In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
import os
import uuid
import numpy as np
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from transformers import AutoTokenizer, AutoModel
# import pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

c:\Users\lycha\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PINECONE_API_KEY ="c5190fed-7028-4d8d-9417-b67ca37c7050"
PINECONE_API_ENV = 'us-east-1'
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV

In [3]:
#Extract data from the PDF
def load_pdf(data):
   loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5039


In [27]:
# text_chunks

[Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf', 'page': 0, 'text': 'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nN-S4', metadata={'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf', 'page': 1, 'text': 'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nN-S4'}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,

In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
#embeddings

In [12]:
query_result = embeddings.embed_query("hello world")
print("length: ", len(query_result))

length:  384


In [26]:
#query_result

In [14]:
# Initialize Pinecone client
pinecone_client = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV )

index_name = 'medical-chatbot'
# Create or get an index
index = pinecone_client.Index(index_name)


# Create the HuggingFaceEmbeddings object
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


# Creating Embeddings for Each of the Text Chunks & storing
# metadata = {}
# for t in text_chunks:
#     embedding = embeddings.embed_query(t.page_content)
#     metadata["text"] = t.page_content
#     index.upsert(vectors=[
#         {
#             "id": str(uuid.uuid4()),
#             "values": embedding,
#             "metadata": metadata
#         }
#     ])

In [29]:

# If we already have an index we can load it like this
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings,index_name=index_name)
query = "What are Allergies"

docs= docsearch.similarity_search(query, k=3)

print("Result",docs)


Result [Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an irritation\nof the membranes of the nose by airborne particles orchemicals. These membranes make mucus. When irritat-ed, they can also grow polyps. The nose is not only a pas-sageway for air to reach the lungs; it also provides theconnection between the sinuses and the outside world.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2315Nasal polyps'), Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an irritation\nof the membranes of the nose by airborne particles orchemicals. These membranes make mucus. When irritat-ed, they can also grow polyps. The nose is not only a pas-sageway for air to reach the lungs; it also provides theconnection between the sinuses and the outside world.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2315Nasal polyps', metadata={'page': 21.0, 'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}), Document(page_content='ry allergies . Hay fever (allergic rhinitis ) is an 

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know. don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context",  "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
llm_model = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})


In [31]:

qa = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever = docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#write me code 

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [ ]:
while True:
    user_input = input(f"Input Prompt:") 
    result = qa({"query": user_input})
    print("Response : ", result["result"])